---
title: Getting daily precipitation from ECMWF Climate Data Store
short_title: Getting CDS precipitation data
---

In this notebook we show how to download daily precipitation data from the European Centre for Medium-Range Weather Forecasts (ECMWF). The ECMWF hosts a large amount of datasets on their [Climate Data Store (CDS)](https://cds.climate.copernicus.eu/datasets) platform. This includes various reanalysis datasets on a large number of weather phenomenon, as well as global forecasting products. The full list of available datasets can be found [on their datasets page](https://cds.climate.copernicus.eu/datasets).

## Manually downloading the data

The first thing we have to do is create a ECMWF user, which gives you free access to the CDS catalog:
- https://www.ecmwf.int/user/login

For this particular excercise, we will use the dataset: 
- ["ERA5-Land hourly data from 1950 to present"](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land)

If you click on the "Dataset" tab, you will see something like this:

![CDS Download Page](./images/ecmwf-cds-download-page.png)

### Variable

In the category "Wind, Pressure and Precipitation", select "Total precipitation" to download.

### Year and Month

- Choose the year and month for which you want to download data.

### Day

- Click "Select all" to download for all available dates of the selected month.

### Time

This dataset contains hourly measurements, and we need all of them to compute daily statistics later on. 

- Click "Select all".

### Geographical Area

To avoid downloading data for the entire world, which can be quite large, we want to limit the download to only the country or region that we are interested in. 

- Click "Sub-region extraction".
- Manually set the minimum and maximum coordinate extent of the region of interest. If you don't know how to set these, there's a simple online tool that can help:
    - Go to http://bboxfinder.com
    - Zoom in to the region of interest.
      ![Screenshot of bboxfinder.com](./images/bboxfinder.png)
    - Copy and insert the coordinates from the bottom of the page under "Map", which are shown in the order of "West, South, East, North".

### Data Format

- Select "NetCDF4"

### Download format

- Select "Zip" to reduce the size of the download file, but either option is fine.

### Terms of Use

- **This step is important**: Before you can download you have to log in with your user and manually accept the terms of reference for this dataset. This is only needed once for each dataset.

### Download the data

If you have followed the previous steps you should be able to:

- Click the "Submit Form" button towards the bottom of the page.
- You'll be taken to a page where you can see the status of your download request.
- Once the "Status" changes to "Complete", you can click "Download" to download your requested file to you computer.


At this point you have successfully retrieved a NetCDF file (`.nc` extension) containing total precipitation data for a region of interest, which can be used for further processing using the other guides and tutorials on this website, like the [Notebook tutorial for Aggregating data from NetCDF files](../aggregation/earthkit-netcdf.ipynb). 

## Automatically downloading the data

You probably noticed in the previous section, that manually downloading data is limited to a single month at a time, and requires a few manual steps for each time. For longer timeseries data, this can be time-consuming and slow to manually download each file. 

Fortunately, the `earthkit.data` package includes a way to programmatically retreive the data from the CDS API. 

In [1]:
import earthkit.data

### Authenticating with your ECMWF user

Before we can download the dataset programmatically, we need to authenticate using our user credentials:

- ...

### Constructing the request query

Next, to obtain the correct parameters to use for this, you can return to the data request page you filled out earlier. 

- At the bottom of the page, click "Show API Request Code" in the "API Request" section.
- This should show something like this:

    ![Screenshot of CDS API request code](./images/ecmwf-cds-api-request-params.png)

        import cdsapi

        dataset = "reanalysis-era5-land"
        request = {
            "variable": ["total_precipitation"],
            "year": "2025",
            "month": "09",
            "day": [
                "01", "02", "03",
                "04", "05", "06",
                "07", "08", "09",
                "10", "11", "12",
                "13", "14", "15",
                "16", "17"
            ],
            "time": [
                "00:00", "01:00", "02:00",
                "03:00", "04:00", "05:00",
                "06:00", "07:00", "08:00",
                "09:00", "10:00", "11:00",
                "12:00", "13:00", "14:00",
                "15:00", "16:00", "17:00",
                "18:00", "19:00", "20:00",
                "21:00", "22:00", "23:00"
            ],
            "data_format": "netcdf",
            "download_format": "unarchived",
            "area": [-9.223115, 26.677001, -17.666611, 43.387206]
        }
        
        client = cdsapi.Client()
        client.retrieve(dataset, request).download()

Earthkit uses `cdsapi` in the background, so we can copy these parameter values to fill in the parameters required by `earthkit`:

In [6]:
data = earthkit.data.from_source("cds",
    "reanalysis-era5-land",
    variable=["total_precipitation"], # you can also include additional variables here to include multiple variables in the same download file
    product_type="reanalysis",
    area=[-9.223115, 26.677001, -17.666611, 43.387206], #N,W,S,E
    date="2012-05-10",
    time="12:00"
    ...need to figure out the correct earthit syntax here
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1498663408.py, line 7)

### Inspect the data

Once the data has finished downloaded, you can then inspect the data: 

In [7]:
data

NameError: name 'data' is not defined

### Save to file

Finally, save the data to file:

In [8]:
data.save(...)

NameError: name 'data' is not defined

### How to download many months? 

One of the benefits of using earthkit to download CDS data, is that we can loop through the months we are interested in and download all with a single script. Here's one possible way of automatic this process: 